# Using L3Net (Cheng et. al. 2020)

The network below can act as a residual block, as the output dimension matches the input dimension.

Note that when instantiating the L3net from its class, we need to pre-specify the adjacency matrix of the graph.

Each row of the input tensor is a nodal feature matrix of size $(V,C)$, where $V$ is the number of node and $C$ is input feature dimension

In [ ]:
import os
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [1]:
from L3net import GraphConv_Bases
import torch.nn as nn
import torch_geometric as pyg
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class input_tranpose(nn.Module):
    def __init__(self, dim0=1, dim1=2):
        super().__init__()
        self.dim0 = dim0
        self.dim1 = dim1

    def forward(self, x):
        return torch.transpose(x, self.dim0, self.dim1)

layers = []
for i in range(1):
    order_list = [0, 1, 2]
    C = 9 # Input feature dimension
    dim = 20
    # three node graph
    edge_index_tmp = torch.tensor(
        [[0, 1, 1, 2, 0, 1, 2], [1, 0, 2, 1, 0, 1, 2]])
    A_ = pyg.utils.to_dense_adj(edge_index_tmp)[
        0].to(device)  # For L3net
    act = nn.ReLU()
    trans = input_tranpose(1, 2)
    layers.append(trans)
    layers.append(GraphConv_Bases(C, dim, A_, order_list=order_list))
    layers.append(act)
    layers.append(GraphConv_Bases(dim, dim, A_, order_list=order_list))
    layers.append(trans)
    layers.append(act)
    layers.append(nn.Linear(dim, dim))
    layers.append(act)
    layers.append(nn.Linear(dim, C))
model = nn.Sequential(*layers)

In [3]:
model

Sequential(
  (0): input_tranpose()
  (1): GraphConv_Bases(
    (coeff_conv): Conv1d(27, 20, kernel_size=(1,), stride=(1,), bias=False)
  )
  (2): ReLU()
  (3): GraphConv_Bases(
    (coeff_conv): Conv1d(60, 20, kernel_size=(1,), stride=(1,), bias=False)
  )
  (4): input_tranpose()
  (5): ReLU()
  (6): Linear(in_features=20, out_features=20, bias=True)
  (7): ReLU()
  (8): Linear(in_features=20, out_features=9, bias=True)
)

In [4]:
N, V = 100, 3
x = torch.randn(N, V, C).to(device)
y = model(x)
x.shape

torch.Size([100, 3, 9])

In [5]:
y.shape

torch.Size([100, 3, 9])